In [37]:
import tensorflow as tf
import pandas as pd
import ta
import warnings
import numpy as np

warnings.filterwarnings("ignore")


In [38]:
if tf.test.gpu_device_name() != "/device:GPU:0":
    print("WARNING: GPU device not found.")
else:
    print("SUCCESS: Found GPU: {}".format(tf.test.gpu_device_name()))


SUCCESS: Found GPU: /device:GPU:0


In [39]:
policy = tf.keras.mixed_precision.Policy("mixed_float16")
tf.keras.mixed_precision.experimental.set_policy(policy)


In [51]:
df = pd.read_parquet("2021ETHdata.parquet")
# df = ta.add_all_ta_features(
#     df,
#     open="open",
#     high="high",
#     low="low",
#     close="close",
#     volume="volume",
#     fillna=True,
# )


In [52]:
n = len(df)
train_df = df[0 : int(n * 0.7)]
val_df = df[int(n * 0.7) : int(n * 0.9)]
test_df = df[int(n * 0.9) :]

x_train = train_df.drop("close", axis=1)
y_train = train_df.loc[:, ["close"]]
x_val = val_df.drop("close", axis=1)
y_val = val_df.loc[:, ["close"]]
x_test = test_df.drop("close", axis=1)
y_test = test_df.loc[:, ["close"]]


In [70]:
def create_dataset(x, y):
    ds = tf.keras.utils.timeseries_dataset_from_array(
        data=x, targets=y, sequence_length=1, batch_size=24
    )
    return ds


train_ds = create_dataset(x_train, y_train)
val_ds = create_dataset(x_val, y_val)
test_ds = create_dataset(x_test, y_test)


In [71]:
norm = tf.keras.layers.Normalization(axis=None)
norm.adapt(x_train)

In [72]:
norm(x_train)

<tf.Tensor: shape=(8340, 4), dtype=float16, numpy=
array([[nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan],
       ...,
       [nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan]], dtype=float16)>

In [45]:
def compile_and_fit(model, ds):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3)

    model.compile(
        loss=tf.losses.MeanSquaredError(),
        optimizer=tf.optimizers.Adam(),
        metrics=[tf.metrics.MeanAbsoluteError()],
    )
    history = model.fit(
        ds,
        epochs=20,
        validation_data=val_ds,
        callbacks=[early_stopping],
    )
    return history


In [48]:
inputs = tf.keras.Input(shape=(None, 90))
x = norm(inputs)
# x = tf.keras.layers.Reshape((-1, 90))(x)
x = tf.keras.layers.GRU(32, return_sequences=True, kernel_regularizer='l2')(x)
# x = tf.keras.layers.Dense(32)(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(inputs, outputs)


In [49]:
performance = {}

performance['gru'] = compile_and_fit(model, train_ds)

Epoch 1/20
348/348 [==============================] - 4s 9ms/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 2/20
  1/348 [..............................] - ETA: 5s - loss: nan - mean_absolute_error: nan

KeyboardInterrupt: 

In [ ]:
model.evaluate(test_ds)

In [ ]:
for x in test_ds.take(1):
    print(x)

In [ ]:
example_window = tf.stack